In [1]:
import trimesh
import numpy as np
import os.path as osp
import os
from shapeglot.simple_utils import unpickle_data
from multiprocessing import Pool
import h5py
import json
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
shapeglot_path = './data/main_data_for_chairs/images/shapenet/03001627'
shapenet_path = '/home/ubuntu/datasets/03001627'

In [65]:
# Load game-data
data_name = 'game_data.pkl'
top_data_dir = 'data/main_data_for_chairs'
game_data, word_to_int, int_to_word, int_to_sn_model, sn_model_to_int, sorted_sn_models =\
unpickle_data(osp.join(top_data_dir, data_name))

In [4]:
def pcd_sample(mesh_dir_path):
    f = os.path.join(mesh_dir_path, 'model.obj')
    mesh = trimesh.load(f, force='mesh')
    samples, face_index = trimesh.sample.sample_surface_even(mesh, 10000, radius=0.01)
    if samples.shape[0] < 2048:
        print(mesh,"have lower than 2048 points")
        samples, face_index = trimesh.sample.sample_surface(mesh, 10000)
    assert samples.shape[0] >= 2048
    np.save(os.path.join(mesh_dir_path, 'pcd'), samples)

In [5]:
shapeglot_list = os.listdir(shapeglot_path)
shapenet_list = os.listdir(shapenet_path)

In [6]:
set(shapenet_list) == set(shapeglot_list)

True

In [7]:
def sample_and_save(sn_model_id, num_points=2048):
    mesh_obj_path = osp.join(shapenet_path, sn_model_id, 'model.obj')
    mesh = trimesh.load(mesh_obj_path, force='mesh')
    samples, face_index = trimesh.sample.sample_surface_even(mesh, 10000, radius=0.001)
    if samples.shape[0] < num_points:
        samples, face_index = trimesh.sample.sample_surface(mesh, num_points)
    samples = samples[:num_points,:]
    assert samples.shape == (num_points, 3)
    np.save(osp.join(shapenet_path, sn_model_id, 'pcd'), samples)
    int_id = sn_model_to_int[sn_model_id]
    return samples, int_id

In [8]:
### Sampling point clouds form meshes ###
pool = Pool(processes=8)
results = pool.map(sample_and_save, sorted_sn_models)

failed to load image
Traceback (most recent call last):
  File "/home/ubuntu/venv/lib/python3.8/site-packages/trimesh/exchange/obj.py", line 324, in parse_mtl
    file_data = resolver.get(split[1])
  File "/home/ubuntu/venv/lib/python3.8/site-packages/trimesh/resolvers.py", line 84, in get
    with open(os.path.join(self.parent, name.strip()), 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/datasets/03001627/1006be65e7bc937e9141f9b58470d646/./models/untitled/texture.jpg'
failed to load image
Traceback (most recent call last):
  File "/home/ubuntu/venv/lib/python3.8/site-packages/trimesh/exchange/obj.py", line 324, in parse_mtl
    file_data = resolver.get(split[1])
  File "/home/ubuntu/venv/lib/python3.8/site-packages/trimesh/resolvers.py", line 84, in get
    with open(os.path.join(self.parent, name.strip()), 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/datasets/03001627/1006be65e7bc937e9141f9b58470d646/./models/un

In [13]:
print(len(results))
assert len(results) == len(sorted_sn_models)

4511


(2048, 3)

In [24]:
def read_point_cloud(sn_id):
    pcd_path = os.path.join(shapenet_path, sn_id, 'pcd.npy')
    assert os.path.exists(pcd_path)
    pc = np.load(pcd_path)
    int_id = sn2int[sn_id]
    return pc, int_id

In [25]:
pool = Pool(processes=8)
sn_list = [sn_id for sn_id in sn2int.keys()]
results = pool.map(read_point_cloud, sn_list)

In [16]:
results=np.array(results)
pcs = np.array(list(results[:,0])).astype(np.float32)
ids = np.array(results[:,1]).astype('int64')
print(pcs.shape, pcs.dtype, ids.shape, ids.dtype)

(4511, 2048, 3) float32 (4511,) int64


In [19]:
out_file = os.path.join('/home/ubuntu/datasets/','shapenet_chairs_only_in_game.h5')
with h5py.File(out_file, 'w') as f:
    f['data'] = pcs
    f['int_ids'] = ids        
    print("Saved '{}'.".format(out_file))

Saved '/home/ubuntu/datasets/shapenet_chairs_only_in_game.h5'.


# Preprocessing PartAnnotation

In [3]:
pc_part_anno_dir = "/home/ubuntu/datasets/full_shapenet_seg/03001627"

In [4]:
sn_list = [sn_id for sn_id in sn_model_to_int.keys()]

In [110]:
def read_point_cloud_part_anno(sn_id):
    pcd_path = osp.join(pc_part_anno_dir, f"{sn_id}.txt")
    pc = np.loadtxt(pcd_path)[:2048,[0,1,2,-1]]
    assert pc.shape[0] == 2048
    int_id = sn_model_to_int[sn_id]
    return pc, int_id

In [105]:
no_in_part_dataset_list = [] # 2038: doesn't exist in part datasets
exist_list = [] #2473
for sn in sn_list: #sn_list: 4511 
    pcd_path = osp.join(pc_part_anno_dir, f"{sn}.txt")
    if not os.path.exists(pcd_path):
        no_in_part_dataset_list.append(sn)
    else:
        exist_list.append(sn)

In [ ]:
with open(osp.join(top_data_dir, "no_in_part_dataset_list.txt"), "w") as f:
    for x in no_in_part_dataset_list:
        f.write(f"{x} ")

In [27]:
chair_a = np.array(game_data.chair_a)
chair_b = np.array(game_data.chair_b)
chair_c = np.array(game_data.chair_c)

In [33]:
chairs = np.array(game_data[["chair_a", "chair_b", "chair_c"]])

In [53]:
no_in_part_dataset_set = set(no_in_part_dataset_int_list)
for i in range(10):
    chat_set = set(chairs[i])
    if chat_set.isdisjoint(no_in_part_dataset_set):
        pass # mask 1
    else:
        pass #mask 0 <- have at least one chair that doesn't exist in part dataset, so should be excluded.
    

False
True
False
False
False
False
True
False
False
True


In [70]:
from shapeglot.in_out.rnn_data_preprocessing import prepare_rnn_dataset
geo_ids, labels, padded_text, seq_len, mask = prepare_rnn_dataset(game_data, max_seq_len=33, only_correct=True, drop_too_long=True,
                    only_easy=True, part_anno=True)

In [74]:
from shapeglot.in_out.rnn_data_preprocessing import make_dataset_for_rnn_based_model

net_data, split_ids, _, net_data_mask = \
make_dataset_for_rnn_based_model(game_data, 
                                 splits=[0.8, 0.1, 0.1], 
                                 max_seq_len=33,
                                 drop_too_long=True,
                                 only_correct=True,
                                 unique_test_geo=True,
                                 replace_not_in_train=True,
                                 geo_condition=None, 
                                 bias_train=False, 
                                 seed=2004,
                                only_easy=True,
                                part_anno=True) 

unique geometries in train/test/val 630 78 79


In [78]:
len(net_data["train"])

4803

In [79]:
net_data2, split_ids2, _, net_data_mask2 = \
make_dataset_for_rnn_based_model(game_data, 
                                 splits=[0.8, 0.1, 0.1], 
                                 max_seq_len=33,
                                 drop_too_long=True,
                                 only_correct=True,
                                 unique_test_geo=True,
                                 replace_not_in_train=True,
                                 geo_condition=None, 
                                 bias_train=False, 
                                 seed=2004,
                                only_easy=True,
                                part_anno=False) 

unique geometries in train/test/val 3145 393 393


In [80]:
len(net_data2["train"])

30632

In [ ]:
pool = Pool(processes=8)
results = pool.map(read_point_cloud_part_anno, exist_list)

In [99]:
results=np.array(results)
pcs_with_part_anno = np.array(list(results[:,0])).astype(np.float32)
ids = np.array(results[:,1]).astype('int64')
print(pcs.shape, pcs.dtype, ids.shape, ids.dtype)

array([[-1.8285e-01,  2.0671e-01, -1.4364e-01,  1.2000e+01],
       [-2.1948e-01,  2.4313e-01,  1.1064e-01,  1.2000e+01],
       [-2.3367e-01,  3.8210e-01, -9.5800e-03,  1.2000e+01],
       ...,
       [-1.7554e-01,  2.3929e-01, -1.7521e-01,  1.2000e+01],
       [-1.8071e-01,  2.1226e-01, -9.8980e-02,  1.2000e+01],
       [ 1.4709e-01,  2.4620e-02,  1.7521e-01,  1.3000e+01]])